In [1]:
if True:
    import os
    os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.3.0 pyspark-shell'
    execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))
    import os
    import sys
else:
    os.environ['SPARK_HOME'] = '/usr/lib/spark'
    sys.path.insert(0, '/usr/lib/spark/python/lib/py4j-0.9-src.zip')
    sys.path.insert(0, '/usr/lib/spark/python/')
    sys.path.insert(0, '/usr/local/lib64/python2.7/site-packages')
    sys.path.insert(0,'/usr/local/lib/python2.7/site-packages')
    
    from pyspark import SparkContext
    from pyspark.sql import SQLContext, HiveContext
    
    try: sc = SparkContext()
    except: None    
    sqlc = SQLContext(sc)
    spark = sqlc

In [3]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, MapType
import pyspark.sql.functions as F

In [5]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [6]:
import seaborn as sns

In [7]:
import numpy as np
import pandas as pd
import time
import json

In [8]:
from operator import add
from pyspark.sql.types import *

In [9]:
from pyspark.mllib.recommendation import Rating
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel
from datetime import datetime

# Load data

In [10]:
def convert_uir_tupple(line):
    r = line.split(',')
    return (int(r[0]), int(r[1]), float(r[2]))

def load_train():
    raw = sc.textFile("lab_12/input/train.csv")\
            .filter(lambda x: not x.startswith("userId"))\
            .map(convert_uir_tupple)
    #return raw.toDF(schema=StructType([StructField("userId",  IntegerType()),
    #                                       StructField("movieId", IntegerType()),
    #                                       StructField("rating",  FloatType())])) 
    return raw
  
def convert_ui_tupple(line):
    r = line.split(',')
    return (int(r[0]), int(r[1]))
    
def load_test():    
    raw = sc.textFile("lab_12/input/test.csv")\
            .filter(lambda x: not x.startswith("userId"))\
            .map(convert_ui_tupple)
    #return raw.toDF(schema=StructType([StructField("userId",  IntegerType()),
    #                                   StructField("movieId", IntegerType())]))     
    return raw

In [11]:
train = load_train()

In [7]:
#train = spark.read.load("/recsys/lab12data/train.csv", 
#                        format="com.databricks.spark.csv", 
#                        delimiter=",", 
#                        header=True,
#                        schema=StructType([StructField("userId",  IntegerType()),
#                                           StructField("movieId", IntegerType()),
#                                           StructField("rating",  FloatType())]))

In [12]:
train.count()

10531564

In [13]:
#test = spark.read.load("/recsys/lab12data/test.csv", 
#                        format="com.databricks.spark.csv", 
#                        delimiter=",", 
#                        header=True,
#                        schema=StructType([StructField("userId",  IntegerType()),
#                                           StructField("movieId", IntegerType())]))

In [14]:
test = load_test()

In [15]:
test.count()

10531564

# Eval utils

In [32]:
#def calc_rmse(df, prediction_column="predicted_rating"):
#    mse = df.select(F.pow(F.col(prediction_column) - F.col("rating"), F.lit(2)).alias("err"))\
#            .agg({"err":"mean"}).collect()[0][0]
#    return np.sqrt(mse)    

# ALS on exact ratings

## Prepeare train,validate,test data

In [16]:
def convert_to_als_rating(r):
    return Rating(r[0], r[1], r[2])

In [18]:
train_ratings = train.map(convert_to_als_rating)

In [19]:
train_ratings.first()

Rating(user=1, product=2244, rating=4.5)

In [20]:
train_ratings.cache()

PythonRDD[7] at RDD at PythonRDD.scala:43

## Train

In [60]:
#v1 - used further
#b_numIterations = 15
#b_rank = 25
#b_lambda = 0.1

#v2 - used for i2i similarity
b_numIterations = 15
b_rank = 40
b_lambda = 0.1 

In [61]:
%%time
model = ALS.train(train_ratings, b_rank, b_numIterations, lambda_=b_lambda, seed=19)

CPU times: user 36 ms, sys: 4 ms, total: 40 ms
Wall time: 2min 7s


In [75]:
#model.save(sc,"lab_12/cache/als25_full_ratings.model")
#model.save(sc,"lab_12/cache/als40_full_ratings.model")

In [63]:
def make_predictions_rdd(model, data_rdd, do_remap):
    if do_remap:
        data =  data_rdd.map(lambda x: (x[0], x[1])) #user, item
    else:
        data = data_rdd
    predictions = model.predictAll(data)
    return predictions

In [64]:
def cut_1to5(x):
    if x < 1.:
        return 1.0
    if x > 5:
        return 5.0
    return x

In [66]:
test_predictions = make_predictions_rdd(model, test, do_remap=False)

In [67]:
x = test_predictions.toDF(schema=StructType([StructField("userId",  IntegerType()),
                                           StructField("movieId", IntegerType()),
                                           StructField("rating",  FloatType())]))

In [68]:
x.show(2)

+------+-------+---------+
|userId|movieId|   rating|
+------+-------+---------+
|185012|  21649| 3.922438|
|185012|  17281|3.3251462|
+------+-------+---------+
only showing top 2 rows



In [69]:
res_pd = x.orderBy("userId","movieId").toPandas()

In [70]:
res_pd['rating'] = res_pd.rating.map(cut_1to5)

In [73]:
#res_pd.to_csv("/data/home/taras.svirsky/lab12/res/res_als_r40_ni15.csv", index=False)
#res_pd.to_csv("/data/home/taras.svirsky/lab12/res/res_als_r25_ni15.csv", index=False)